# Erstellung des Baseline Models für den Testdatensatz

### Laden des Testdatensatzes

In [13]:
# Importieren der nötigen Librarys
import pandas as pd
import statsmodels.formula.api as smf


# Einlesen des Trainingdatensatzes
url_train = "https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/train_dataf_knn.csv"
train_data = pd.read_csv(url_train)

print(train_data.shape)
print(train_data.isnull().sum()) # shows number of missing values -> (NaN in numeric arrays, None or NaN in object arrays, NaT in datetimelike)

print(train_data)

(7523, 20)
Datum                    0
KielerWoche              0
Warengruppe              0
Umsatz                   0
Bewoelkung               0
Temperatur               0
Windgeschwindigkeit      0
Wettercode               0
FerienSH                 0
Feiertag                 0
Umschlag                 0
Weihnachtsmarkt          0
Verbraucherpreisindex    0
Regen                    0
Schnee                   0
Wochentag_MDMDFSS        0
Wochenende               0
Jahreszeit_FSHW          0
Temperatur_Kategorie     0
THW_heimspiel            0
dtype: int64
           Datum  KielerWoche  Warengruppe      Umsatz  Bewoelkung  \
0     2013-07-01          0.0          1.0  148.828353         6.0   
1     2013-07-01          0.0          2.0  535.856285         6.0   
2     2013-07-01          0.0          3.0  201.198426         6.0   
3     2013-07-01          0.0          4.0   65.890169         6.0   
4     2013-07-01          0.0          5.0  317.475875         6.0   
...          ...

In [8]:
# Erstellen einer Modellgleichung

#mod1 = smf.ols('Umsatz ~ C(Warengruppe)*C(Feiertag) + C(Warengruppe)*Jahreszeit_FSHW + C(KielerWoche) + C(Wochentag) + C(Wochenende)', data = train_data).fit()
#print(mod1.summary())

#mod2 = smf.ols('Umsatz ~ C(Warengruppe)*C(KielerWoche) + C(Warengruppe)*C(Wochentag) + C(Warengruppe)*C(Wochenende) + C(Wochenende) + C(Wochentag)', data = train_data).fit()
#print(mod2.summary())

#mod3 = smf.ols('Umsatz ~ C(Warengruppe)*C(KielerWoche) + C(Temp) + C(Warengruppe)*C(Wochenende) + C(Wochenende) + C(Wochentag)', data = train_data).fit()
#print(mod2.summary())


In [9]:
# Erstellen einer Modellgleichung für den Testdatensatz gemeinsam:

mod_1 = smf.ols('Umsatz ~ C(Warengruppe)*C(Jahreszeit_FSHW) + Regen + C(Warengruppe)*C(Wochenende) + C(KielerWoche) + C(Wochentag_MDMDFSS) + C(Wochenende) + Jahreszeit_FSHW + Temperatur_Kategorie', data = train_data).fit()
print(mod_1.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     505.6
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:06:51   Log-Likelihood:                -42298.
No. Observations:                7523   AIC:                         8.471e+04
Df Residuals:                    7465   BIC:                         8.511e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [10]:
mod_2 = smf.ols('Umsatz ~ C(Warengruppe)*C(FerienSH) + C(Regen) + C(Warengruppe)*C(Wochenende) + C(KielerWoche) + C(Wochentag_MDMDFSS) + C(Wochenende) + C(Jahreszeit_FSHW) + C(Temperatur_Kategorie)*C(Jahreszeit_FSHW)', data = train_data).fit()
print(mod_2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     428.9
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:06:55   Log-Likelihood:                -42440.
No. Observations:                7523   AIC:                         8.501e+04
Df Residuals:                    7458   BIC:                         8.546e+04
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
                                                                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [11]:
mod_3 = smf.ols('Umsatz ~ C(Warengruppe)*C(KielerWoche) + C(Warengruppe)*C(Wochenende) + C(KielerWoche) + C(Jahreszeit_FSHW)', data = train_data).fit()
print(mod_3.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     545.6
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:07:00   Log-Likelihood:                -43041.
No. Observations:                7523   AIC:                         8.617e+04
Df Residuals:                    7481   BIC:                         8.646e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [12]:
# Einlesen des Trainingdatensatzes ohne Lücken
url_train = "https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/train_data_no_missing_data.csv"
train_data_nmd = pd.read_csv(url_train)

# mod_4 = smf.ols('Umsatz ~ Warengruppe + Temperatur_Kategorie:C(Jahreszeit_FSHW) + KielerWoche + Bewoelkung + Windgeschwindigkeit + Wettercode + FerienSH + Feiertag + Heim_Auswärts + Umschlag + Weihnachtsmarkt + Regen + Wochentag_MDMDFSS + Wochenende', data = train_data_nmd).fit()

mod_4 = smf.ols('Umsatz ~ Warengruppe + Temperatur_Kategorie:C(Jahreszeit_FSHW) + FerienSH + Feiertag + Weihnachtsmarkt + Wochenende', data = train_data_nmd).fit()

print(mod_4.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     65.93
Date:                Wed, 19 Jun 2024   Prob (F-statistic):          6.26e-177
Time:                        06:07:02   Log-Likelihood:                -47835.
No. Observations:                7523   AIC:                         9.570e+04
Df Residuals:                    7508   BIC:                         9.580e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

## Weitere Modelle (Fabian)

In [21]:
mod_4 = smf.ols('Umsatz ~ C(Warengruppe) + C(Temperatur_Kategorie)*C(Jahreszeit_FSHW) + C(Warengruppe)*C(Wochenende) + C(Wochentag_MDMDFSS) + C(Warengruppe)*C(Wochentag_MDMDFSS) + C(Regen)*C(Jahreszeit_FSHW) + C(Windgeschwindigkeit)*C(Jahreszeit_FSHW) + C(Bewoelkung)*C(Jahreszeit_FSHW) + C(Bewoelkung)*C(Temperatur_Kategorie)', data = train_data).fit()
print(mod_4.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.964
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     50.44
Date:                Mon, 10 Jun 2024   Prob (F-statistic):           1.86e-61
Time:                        12:38:06   Log-Likelihood:                -879.71
No. Observations:                 178   AIC:                             1883.
Df Residuals:                     116   BIC:                             2081.
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

### Einlesen des Trainingsdatensatzes + Test des Modells

In [ ]:
# Einlesen des Trainingdatensatzes
url_train = "https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/train_data.csv"
train_data = pd.read_csv(url_train)


print(mod_4.summary())